<a href="https://colab.research.google.com/github/SriNithin965/cognizant_internship/blob/main/multi_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/SriNithin965/cognizant_internship/main/iris%20.csv')

In [ ]:
df.head(10)

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
columns = ['encoded_class','sepal_length','sepal_width','petal_length','petal_width']

In [ ]:
df['encoded_class'] = le.transform(df['class'])

In [ ]:
np.random.seed(5)
l = list(df.index)
np.random.shuffle(l)
df = df.iloc[l]

In [ ]:
rows = df.shape[0]
train = int(.7 * rows)
test = rows-train

In [ ]:
rows, train, test

(150, 105, 45)

In [ ]:
df[:train].to_csv('iris_train.csv'
                          ,index=False,header=False
                          ,columns=columns)

In [ ]:
df[train:].to_csv('iris_validation.csv'
                          ,index=False,header=False
                          ,columns=columns)

<h1>LOCAL MODE OF XGBOOST IMPLEMENTATION</h1>

In [ ]:
!pip install xgboost==1.2

     |████████████████████████████████| 148.9 MB 68.3 MB/s            


In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import xgboost as xgb

from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

Matplotlib is building the font cache; this may take a moment.


In [ ]:
train_file = 'iris_train.csv'
validation_file = 'iris_validation.csv'

In [ ]:
columns=['encoded_class','sepal_length','sepal_width','petal_length','petal_width']

In [ ]:
labels=[0,1,2]
classes = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
le = preprocessing.LabelEncoder()
le.fit(classes)

LabelEncoder()

In [ ]:
df_train = pd.read_csv(train_file,names=columns)
df_validation = pd.read_csv(validation_file,names=columns)

In [ ]:
X_train = df_train.iloc[:,1:] # Features: 1st column onwards 
y_train = df_train.iloc[:,0].ravel() # Target: 0th column

X_validation = df_validation.iloc[:,1:]
y_validation = df_validation.iloc[:,0].ravel()

In [ ]:
classifier = xgb.XGBClassifier(objective="multi:softmax",
                               num_class=3,
                               n_estimators=100)

In [ ]:
classifier.fit(X_train,
               y_train,
               eval_set = [(X_train, y_train), (X_validation, y_validation)],
               eval_metric=['mlogloss'],
               early_stopping_rounds=10)

[0]	validation_0-mlogloss:0.73876	validation_1-mlogloss:0.74994
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:0.52787	validation_1-mlogloss:0.55401
[2]	validation_0-mlogloss:0.38960	validation_1-mlogloss:0.42612
[3]	validation_0-mlogloss:0.29429	validation_1-mlogloss:0.34328
[4]	validation_0-mlogloss:0.22736	validation_1-mlogloss:0.29000
[5]	validation_0-mlogloss:0.17920	validation_1-mlogloss:0.24961
[6]	validation_0-mlogloss:0.14403	validation_1-mlogloss:0.22234
[7]	validation_0-mlogloss:0.11664	validation_1-mlogloss:0.20338
[8]	validation_0-mlogloss:0.09668	validation_1-mlogloss:0.18999
[9]	validation_0-mlogloss:0.08128	validation_1-mlogloss:0.18190
[10]	validation_0-mlogloss:0.06783	validation_1-mlogloss:0.17996
[11]	validation_0-mlogloss:0.05794	validation_1-mlogloss:0.18029
[12]	validation_0-mlogloss:0.05011	validation_1-mlogloss:0.1830

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_class=3, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
eval_result = classifier.evals_result()

In [ ]:
df = pd.read_csv(validation_file,names=columns)

In [ ]:
X_test = df.iloc[:,1:]
print(X_test[:5])

   sepal_length  sepal_width  petal_length  petal_width
0           5.8          2.7           4.1          1.0
1           4.8          3.4           1.6          0.2
2           6.0          2.2           4.0          1.0
3           6.4          3.1           5.5          1.8
4           6.7          2.5           5.8          1.8


In [ ]:
result = classifier.predict(X_test)

In [ ]:
df['predicted_class'] = result

In [ ]:
print(classification_report(
    df['encoded_class'],
    df['predicted_class'],
    labels=labels,
    target_names=classes))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        16
Iris-versicolor       0.91      0.91      0.91        11
 Iris-virginica       0.94      0.94      0.94        18

       accuracy                           0.96        45
      macro avg       0.95      0.95      0.95        45
   weighted avg       0.96      0.96      0.96        45



<h1>XGBOOST USING THE CONTAINER</h1>

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [ ]:
region = boto3.Session().region_name
session = sagemaker.Session()
bucket = session.default_bucket()
role = get_execution_role()
sm = boto3.Session().client(service_name = 'sagemaker',region_name = region,
                           )

In [ ]:
prefix = 'iris'

In [ ]:
bucket_name = bucket
training_file_key = 'iris/iris_train.csv'
validation_file_key = 'iris/iris_validation.csv'

s3_model_output_location = r's3://{0}/iris/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name,training_file_key)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name,validation_file_key)

In [ ]:
def write_to_s3(filename,key):
        return session.upload_data(path=filename, key_prefix=key)

In [ ]:
write_to_s3('iris_train.csv','iris')
write_to_s3('iris_validation.csv','iris')

's3://sagemaker-us-west-1-313830654669/iris/iris_validation.csv'

In [ ]:
use_spot_instances = True
max_run = 3600 # in seconds
max_wait = 7200 if use_spot_instances else None # in seconds

job_name = 'xgboost-iris-v1'

checkpoint_s3_uri = None

if use_spot_instances:
    checkpoint_s3_uri = f's3://{bucket_name}/iris/checkpoints/{job_name}'
    
print (f'Checkpoint uri: {checkpoint_s3_uri}') 

Checkpoint uri: s3://sagemaker-us-west-1-313830654669/iris/checkpoints/xgboost-iris-v1


In [ ]:
container = sagemaker.image_uris.retrieve("xgboost",session.boto_region_name,version="1.2-2")

print (f'Using XGBoost Container {container}')

Using XGBoost Container 746614075791.dkr.ecr.us-west-1.amazonaws.com/sagemaker-xgboost:1.2-2


In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=s3_model_output_location,
    sagemaker_session=session,
    base_job_name = job_name,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri)

In [ ]:
estimator.set_hyperparameters(max_depth=5,
                              objective="multi:softmax",
                              eval_metric="mlogloss",
                              num_class=3,
                              num_round=100,
                              early_stopping_rounds=10)

In [ ]:
# content type can be libsvm or csv for XGBoost
training_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_training_file_location,
    content_type='csv',
    s3_data_type='S3Prefix')

validation_input_config = sagemaker.session.TrainingInput(
    s3_data=s3_validation_file_location,
    content_type='csv',
    s3_data_type='S3Prefix'
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

In [ ]:
estimator.fit(data_channels)

2022-06-15 09:43:41 Starting - Starting the training job...
2022-06-15 09:44:05 Starting - Preparing the instances for trainingProfilerReport-1655286221: InProgress
............
2022-06-15 09:46:05 Downloading - Downloading input data
2022-06-15 09:46:05 Training - Downloading the training image......
2022-06-15 09:47:06 Training - Training image download completed. Training in progress..[2022-06-15 09:47:08.604 ip-10-0-150-244.us-west-1.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-06-15:09:47:08:INFO] Imported framework sagemaker_xgboost_container.training
[2022-06-15:09:47:08:INFO] Failed to parse hyperparameter eval_metric value mlogloss to Json.
Returning the value itself
[2022-06-15:09:47:08:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2022-06-15:09:47:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-06-15:09:47:08:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-06-15:

<h1>END POINT CREATION </h1>

In [ ]:
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.m5.xlarge',
                             endpoint_name = job_name)

-----!

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
predictor.serializer = CSVSerializer()

In [ ]:
df_all = pd.read_csv('iris_validation.csv',
                     names=['encoded_class','sepal_length','sepal_width','petal_length','petal_width'])

In [ ]:
arr_test = df_all[['sepal_length', 'sepal_width', 'petal_length','petal_width']].values

In [ ]:
result = predictor.predict(arr_test[:2])

In [ ]:
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    result = result.decode("utf-8")
    result = result.split('\n')
    result = result[0:len(result)-1]
    print (arr.shape)
    print(result)
    predictions+=[int(float(r)) for r in result]

(5, 4)
['1.0', '0.0', '1.0', '2.0', '2.0']
(5, 4)
['2.0', '2.0', '0.0', '0.0', '2.0']
(5, 4)
['2.0', '0.0', '1.0', '0.0', '0.0']
(5, 4)
['2.0', '1.0', '2.0', '0.0', '2.0']
(5, 4)
['0.0', '2.0', '0.0', '1.0', '0.0']
(4, 4)
['2.0', '2.0', '0.0', '1.0']
(4, 4)
['2.0', '2.0', '1.0', '0.0']
(4, 4)
['1.0', '0.0', '0.0', '1.0']
(4, 4)
['2.0', '0.0', '2.0', '0.0']
(4, 4)
['1.0', '2.0', '2.0', '1.0']


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])

LabelEncoder()

In [ ]:
df_all['class'] = le.inverse_transform(df_all.encoded_class)

In [ ]:
df_all['predicted_class']=le.inverse_transform(predictions)

In [ ]:
import sklearn.metrics as metrics
print(metrics.classification_report(df_all['class'], df_all['predicted_class']))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        16
Iris-versicolor       0.91      0.91      0.91        11
 Iris-virginica       0.94      0.94      0.94        18

       accuracy                           0.96        45
      macro avg       0.95      0.95      0.95        45
   weighted avg       0.96      0.96      0.96        45



In [ ]:
session.delete_endpoint(job_name)

<h1>BOTH MODELS YIELD ALMOST SAME RESULTS</h1>